### Import libraries

In [1]:
import pandas as pd
import pathlib
import re
import numpy as np

import sys
sys.path.append("../utils/")
from locate_utils import get_training_locations, get_control_locations

### Specify paths

In [2]:
annotations_path = pathlib.Path("idr0013-screenA-annotation.csv.gz")
features_path = pathlib.Path("trainingset_2007_06_21.dat")

locations_dir = pathlib.Path("locations/")
locations_dir.mkdir(exist_ok=True, parents=True)

### Find/save training locations

In [3]:
training_save_path = pathlib.Path(f"{locations_dir}/training_locations.tsv")

training_data_locations = get_training_locations(features_path, annotations_path)
training_data_locations.to_csv(training_save_path, sep="\t")

print(training_data_locations.shape)
training_data_locations.head()

Image from LT0002_17, 36 not in IDR
Image from LT0155_02, 24 not in IDR
Image from LT0155_02, 24 not in IDR
Image from LT0078_27, 244 not in IDR
Image from LT0002_17, 36 not in IDR
Image from LT0155_02, 24 not in IDR
Image from LT0002_17, 36 not in IDR
Image from LT0002_17, 36 not in IDR
Image from LT0155_02, 24 not in IDR
Image from LT0111_45, 53 not in IDR
Image from LT0002_17, 36 not in IDR
Image from LT0002_17, 36 not in IDR
Image from LT0111_45, 53 not in IDR
Image from LT0111_45, 53 not in IDR
Image from LT0078_27, 244 not in IDR
Image from LT0111_45, 53 not in IDR
Image from LT0078_27, 244 not in IDR
Image from LT0002_17, 36 not in IDR
Image from LT0111_45, 53 not in IDR
Image from LT0111_45, 53 not in IDR
Image from LT0155_02, 24 not in IDR
Image from LT0002_17, 36 not in IDR
(50, 5)


,Plate,Well,Well Number,Frames,Original Gene Target
0,LT0098_13,A21,21,"77,21,27,33",ENSG00000110675
1,LT0132_31,C5,53,"33,75,37,42,51,63",ENSG00000123416
2,LT0093_17,E18,114,"45,72,46,88,83,79,87",ENSG00000174442
3,LT0094_04,N7,319,"7,66,61,89,73,5,93,30",ENSG00000177426
4,LT0026_22,K18,258,"85,37,25,95,67",LCK


### Find/save negative control locations

In [4]:
negative_control_save_path = pathlib.Path(f"{locations_dir}/negative_control_locations.tsv")

negative_control_locations = get_control_locations(annotations_path, "negative", 0)
negative_control_locations.to_csv(negative_control_save_path, sep="\t")

print(negative_control_locations.shape)
negative_control_locations.head()

(3853, 5)


,Plate,Well,Well Number,Original Gene Target,Frames
0,LT0001_02,A15,15,negative control,43
1,LT0001_02,B2,26,negative control,46
2,LT0001_02,C15,63,negative control,52
3,LT0001_02,D2,74,negative control,31
4,LT0001_02,M16,304,negative control,34


### Find/save positive control locations

In [5]:
positive_control_save_path = pathlib.Path(f"{locations_dir}/positive_control_locations.tsv")

positive_control_locations = get_control_locations(annotations_path, "positive", 1)
positive_control_locations.to_csv(positive_control_save_path, sep="\t")

print(positive_control_locations.shape)
positive_control_locations.head()

(2890, 5)


,Plate,Well,Well Number,Original Gene Target,Frames
0,LT0001_02,A4,4,ENSG00000149503,36
1,LT0001_02,C1,49,ENSG00000149503,42
2,LT0001_02,C4,52,ENSG00000149503,43
3,LT0001_09,A4,4,ENSG00000149503,39
4,LT0001_09,C1,49,ENSG00000149503,62
